# MP3 -> MP4

In [ ]:
# convert mp3 into mp4 with static image

In [184]:
STATIC_IMAGE = "esl_static.png"
INPUT_MP3 = "test_demo.mp3"
OUTPUT_MP4 = "test_output.mp4"

In [188]:
!echo $STATIC_IMAGE
!echo $INPUT_MP3
!echo $OUTPUT_MP4

esl_static.png
test_demo.mp3
test_output.mp4


In [189]:
!ffmpeg -loop 1 -framerate 1 -i $STATIC_IMAGE -i $INPUT_MP3 -map 0:v -map 1:a -r 10 -vf "scale='iw-mod(iw,2)':'ih-mod(ih,2)',format=yuv420p" -movflags +faststart -shortest -fflags +shortest -max_interleave_delta 100M $OUTPUT_MP4

ffmpeg version 4.4 Copyright (c) 2000-2021 the FFmpeg developers
  built with Apple clang version 12.0.0 (clang-1200.0.32.29)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/4.4_2 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --disable-indev=jack --enable-avresample --enable-videotoolbox
  libavutil      

# CREATE SUBTITLE

In [111]:
import json
import tqdm
import pandas as pd

In [ ]:
TRANSCRIBED_JSON = "./yt5s.com-5-MINUTE.json"
OUTPUT_SRT = "sub_title.srt"
GROUPED_TIME = 3
DURATION_TIME = 2

In [5]:
# take as input the transcribed job
with open(TRANSCRIBED_JSON, "r") as f:
    data_json = json.loads(f.read())
    
# full transcript  
full_text = data_json["results"]["transcripts"][0]["transcript"]

In [178]:
dump = []
max_iter =  len(data_json["results"]["items"])


for i in range(max_iter):
    try:
        # extract start time
        start_time = data_json["results"]["items"][i]["start_time"]
        # extract end time
        end_time = data_json["results"]["items"][i]["end_time"]
        # transcribed content
        content_transcribed = data_json["results"]["items"][i]["alternatives"][0]["content"]
        # append result in a list
        dump.append([start_time, end_time, content_transcribed])
    except:
        pass

In [163]:
# construction of the dataset
df_subtitles = pd.DataFrame(dump, columns = ["start", "end", "word"])
df_subtitles["start"] = (df_subtitles["start"].astype(float)/GROUPED_TIME).round().astype(int)*GROUPED_TIME
df_subtitles["end"] = df_subtitles["start"] + DURATION_TIME
df_grouped = df_subtitles.groupby("start").agg(lambda x: " ".join(x))
df_grouped.reset_index(inplace=True)

# creation of label (subtitle)
df_grouped["end"] = (df_grouped["start"]).shift(-1)
df_grouped.loc[df_grouped.index[-1], "end"] = df_grouped.loc[df_grouped.index[-1], "start"]
df_grouped["end"] = df_grouped["end"].astype(int)

# creation of timestamp label (subtitle)
df_grouped["ts_start"]=  df_grouped["start"].apply(lambda x: f'00:00:{x:02d},000')
df_grouped["ts_end"]=  df_grouped["end"].apply(lambda x: f'00:00:{x:02d},000')

#  creation of the subtitle arrow time duration
df_grouped["subtitle_line"] = df_grouped["ts_start"] + " --> "+ df_grouped["ts_end"]

In [174]:
# create an output text file
for val in tqdm.tqdm(df_grouped.values):
    with open(OUTPUT_SRT, "a") as f:
        f.write(str(val[0]))
        f.write("\n")
        f.write(str(val[5]))
        f.write("\n")
        f.write(str(val[1]))
        f.write("\n")
        f.write("\n")

100%|██████████| 118/118 [00:00<00:00, 8474.07it/s]
